OverFeat и R-CNN - две работы по детектированию объектов с помощью сверточных сетей, опубликованные в конце 2013 года.

### Принцип работы OverFeat

Идея OverFeat заключается в использовании квадратного скользящего окна разных масштабов. Когда скользящее окно находится на некой позиции, оно ограничивает некий квадратный участок изображения. Сверточная сеть извлекает из данного участка карту признаков, а полносвязная сеть осуществляет классификацию по этой карте признаков. Другая полносвязная сеть решает задачу регрессии, уточняя координаты ограничивающего прямоугольника для найденного объекта. Далее рассмотрим архитектуру более подробно.

Если скользящее окно сдвинуто из позиции A в позицию B, то признаки, извлекаемые сверточными слоями из области C, останутся теми же самыми (если не учитывать краевые эффекты). Поэтому не имеет смысла запускать сверточную сеть целиком для каждой позиции скользящего окна.

<img src="assets/overfeat.jpg" width="250" align="center">

В OverFeat прежде всего выбирается количество масштабов скользящего окна - это гиперпараметр модели. Допустим мы используем 6 масштабов. Для каждого входного изображения создается 6 версий с разными масштабами: от 245х245 до 461х569. Далее для каждой версии однократно запускается сверточная сеть, извлекая карты признаков. В итоге мы получаем 6 карт признаков разного размера: от 17х17х1024 до 35х44х1024. Последняя цифра - количество каналов.

Далее применяется MaxPooling с размером окна 3 и шагом 3, при этом он применяется со всеми возможными смещениями, то есть от 0 до 2 по вертикали и от 0 до 2 по горизонтали. Таким образом, мы получили 9 карт признаков размером 9х9х1024.

Классификатором является полносвязная сеть, которая принимает на вход карту признаков размером 5x5x1024 (и "вытягивает ее в вектор" операцией reshape). Поскольку входной массив имеет по X и Y размер больше, чем 5х5, то классификатор применяется со всеми возможными смещениями.

Все это можно записать следующим numpy-псевдокодом:

In [ ]:
# for example, feature_map has shape (29, 29, 1024)
classifier_outputs = np.zeros((4, 4, 3, 3, 1000))
for pooling_dx in range(3):
    for pooling_dy in range(3):
        pooled_feature_map = MaxPooling2D(feature_map[::dx, ::dy], size=(3, 3), stride=(3, 3))
        # pooled_feature_map has shape (9, 9, 1024)
        for classifier_dx in range(4): # <-- more generally, use (pooled_feature_map.shape[0] - 5) instead of "4"
            for classifier_dy in range(4): # <-- more generally, use (pooled_feature_map.shape[1] - 5) instead of "4"
                probabilities = Classifier(pooled_feature_map[::classifier_dx, ::classifier_dy])
                # probabilities have shape (1000,)
                classifier_outputs[classifier_dx, classifier_dy, pooling_dx, pooling_dy] = probabilities

Иными словами, мы сначала применяем MaxPooling 3х3 со всеми возможными смещениями, а затем применяем классификатор со всеми возможными смещениями. Каждому результату работы классификатора можно сопоставить область на исходном изображении, ему соответствующую ("скользящее окно"). Эта область - квадрат, его масштаб в пикселях фиксирован, а его смещение зависит от того, какие смещения использовались при пулинге и применении классификатора.

Кроме классификатора мы используем и другую "голову" сети - регрессор. Он принимает на вход те же данные, что и классификатор, и предсказывает 4 числа: координаты ограничивающего прямоугольника (bounding box) для объекта на изображении, относительно границ скользящего окна.

Результате применения данной нейросети к изображению можно представить в виде большого набора словарей, где каждый словарь содержит 3 ключа:

1. Вероятности классов (1000 классов = 1000 чисел)
2. Абсолютные координаты скользящего окна
3. Координаты ограничивающего прямоугольника относительно скользящего окна

По этим данным можно рассчитать абсолютные координаты ограничивающего прямоугольника в пикселях. Также можно найти наиболее вероятный класс операцией argmax. В результате мы получим большой набор пар (object, x1, x2, y1, y2).

На этом, однако, задача не заканчивается. Если на изображении присутствует один объект (например, медведь), то существует множество масштабов и положений скользящего окна, при которых медведь попадет в окно и будет классифицирован как медведь. Например, одно окно содержит только голову медведя, другое только тело, и на обоих окнах будет найден медведь. Поэтому полученный набор пар (object, x1, x2, y1, y2) содержит один и тот же объект во множестве экземпляров.

Для поиска одного, наиболее вероятного ограничивающего прямоугольника для каждого найденного объекта нужно как-то объединить все полученные данные. Для этого авторы используют итеративный алгоритм, который на каждом шаге объединяет два наиболее похожих ограничивающих прямоугольника, соответствующих одному и тому же классу. Более подробно алгоритм объединения описан в разделе 4.3 статьи.

На иллюстрации ниже показаны "сырые" данные, выданные OverFeat (для 4 разных масштабов изображения), и результат после их объединения.

<img src="assets/overfeat2.jpg" width="800" align="center">

Авторы детально описывают сверточную архитектуру ("backbone"), использованную в OverFeat, но здесь мы рассматривать ее не будем, поскольку современные архитектуры уже давно ушли от архитектур 2013 года, в которых еще использовались свертки 11х11 и 7х7 (как в [AlexNet]($ImageNet Classification with Deep Convolutional Neural Networks$)).

### Задачи и метрики в ImageNet Recognition Challenge

Авторы OverFeat описывают три типа задач, для которых может использоваться архитектура OverFeat: детектирование, локализация и классификация.

1. **Детектирование.** Согласно терминологии, применяемой в соревновании [ILSVRC 2013](https://www.image-net.org/challenges/LSVRC/2013/index.php) (ImageNet Large Scale Visual Recognition Challenge), задача детектирования заключается в поиске объектов на изображении, то есть указании классов и ограничивающих прямоугольников. Объектов может быть несколько, или не быть ни одного.

2. **Локализация.** Частный случай детектирования, при котором известно, что на изображении находится ровно один интересующий нас объект. Ответом является класс и ограничивающий прямоугольник.

3. **Классификация.** Частный случай локализации, когда нас не интересует ограничивающий прямоугольник, а интересует только класс.

В соревновании ILSVRC для оценки качества классификации и локализации используются метрики top-1 accuracy и top-5 accuracy.

**Метрика top-1 accuracy** дает только один шанс угадать объект. Если это задача классификации, то угадать объект - значит верно определить его класс. Если это задача локализации, то угадать объект - значит во-первых верно определить его класс, а во-вторых определить его ограничивающий прямоугольник с точностью более 50% совпадения. Для этого нужно указать "эталонный" ограничивающий прямоугольник (ground truth), что делает человек-аннотатор при разметке датасета.

**Метрика top-5 accuracy** дает 5 шансов угадать объект. Если с первого раза объект угадан неверно (либо класс, либо прямоугольник), то дается вторая попытка, и так до 5 попыток. Смысл этой метрики в том, что некоторые классы в ImageNet очень похожи (например, некоторые породы собак), и если алгоритм угадал почти верно (например указал одну породу собаки вместо другой), то это тоже можно считать хорошим результатом.

Для оценки качества детектирования применяется метрика **Mean Average Precision (mAP)**, которую мы здесь подробно не рассматриваем.

### Результаты OverFeat

Команда, разработавшая OverFeat, достигла следующих результатов:

- Детектирование: 24.3% mAP
- Локализация: 70.1% top-5 accuracy
- Классификация: 86.4% top-5 accuracy

Для сравнения, на конец 2021 года (8 лет спустя) достигнута точность [98.8% top-5 accuracy](https://paperswithcode.com/sota/image-classification-on-imagenet?metric=Top%205%20Accuracy) на классификации ImageNet.

Практически одновременно с OverFeat появилась другая модель R-CNN, которая достигла точности 31.4% mAP на детектировании ILSVRC 2013. Эту модель мы разберем далее.

### Принцип работы R-CNN

Здесь мы будем рассматривать оригинальную R-CNN (2013). Впоследствии появилось много ее модификаций: [SPP]($Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition$) (2014), [Fast R-CNN]($Fast R-CNN$) (2015), [Faster R-CNN]($Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks$) (2015), [Mask R-CNN]($Mask R-CNN$) (2017).

Работа R-CNN основана на **регионах-кандидатах (region proposals)**. На первом шаге делается поиск около 2000 регионов-кандидатов, которые возможно содержат какой-то объект (пока неизвестной категории). Это может делаться самыми разными способами, в том числе без применения нейронных сетей. В разделе 2.1 авторы перечисляют список известных метододов для поиска регионов-кандидатов и останавливаются на методе selective search.

[Selective search]($Selective Search for Object Recognition$) - это алгоритм, который ищет похожие по цвету и текстуре области пикселей. На примере ниже изображение разбито на группы, каждой группе сопоставлен условный цвет:

<img src="assets/selective_search.jpg" width="800" align="center">

Затем на основе найденных групп составляется список регионов-кандидатов, которые возможно содержат какой-то объект.

<img src="assets/selective_search2.jpg" width="800" align="center">

(изображение из статьи [Selective Search for Object Recognition]($Selective Search for Object Recognition$))

Каждый регион расширяется на некоторое количество пикселей, а затем масштабируется до размера 227х227 (без сохранения соотношения сторон) и обрабатывается нейронной сетью из 5 сверточных и 2 полносвязных слоев. Эта сеть не осуществляет классификацию, а лишь создает вектор признаков.

Сначала мы рассмотрим алгоритм инференса R-CNN, а затем процесс обучения.

**Инференс R-CNN**

Для классификации N классов имеется N SVM (машин опорных векторов). Каждая SVM образатывает вектор признаков и выдает число, означающее степень соответствия объекта соответствующему классу. Для этого считается скалярное произведение вектора признаков с весами SVM. Понятно, что с помощью матричного умножения можно эффективно посчитать ответы сразу всех SVM для данного вектора. Это аналогично работе полносвязного слоя, отличается лишь принцип обучения (полносвязный слой обучается не так, как SVM).

После этого из всех регионов-кандидатов нужно выбрать самые правдоподобные. Это делается для каждого класса независимо. Зафиксируем некий класс C. Будем применять операцию non-maximum supression: если два региона пересекаются более чем на R процентов (R - гиперпараметр), то среди них отбрасывается тот, в котором степень соотвествия классу оказалась меньше. Применив данную операцию много раз, мы получаем финальные ограничивающие прямоугольники для найденных на изображении объектов.

**Обучение R-CNN**

В качестве backbone авторы пробуют применять две сверточные сети (OxfordNet, TorontoNet), подробно рассматривать их архитектуры здесь мы не будем. Сверточная сеть предобучается на задаче классификации ImageNet. Далее сеть дообучается на датасете для детектирования следующим образом:

1. Выходной слой сети заменяется на слой размером N+1, то есть сеть теперь решает задачу классификации на N классов плюс класс "background".
2. Датасет для детектирования является размеченным, то есть для i-го изображения мы знаем метку класса $C_i$ и ограничивающий прямоугольник $B_i$. Из каждого изображения мы извлекаем регионы-кандидаты методом selective search. Каждому региону-кандидату i-го изображения присваивается метка класса: $C_i$, если регион-кандидат совпадает с истинным регионом $B_i$ более чем на 50%, в противном случае меткой класса является "background".
3. Каждый батч собирается случайно из 96 изображений с меткой класса "background" и 32 изображений с метками других классов.
4. Сверточная сеть обучается на этих данных.
5. После обучения выходной слой сети отбрасывается, и теперь для каждого региона сеть будет выдавать вектор признаков (значения на последнем скрытом слое).

SVM-классификаторы для каждого класса обучаются похожим образом. Каждый SVM является бинарным классификатором. Входными данными является вектор признаков, извлеченный обученной сетью из региона-кандидата. Выходными данными является 1 в том случае, если на изображении действительно изображен  объект этого класса, и регион-кандидат совпадает с истинным ограничивающим прямоугольником не менее, чем на 30% (а не 50%, как при обучении CNN). Все данные для обучения SVM не помещаются в память, поэтому применяется метод hard negative mining.

Для уточнения границ регионов-кандидатов, как в и OverFeat, применяется регрессия (bounding box regression). Более подробно останавливаться на деталях архитектуры R-CNN не будем, поскольку это одна из ранних моделей для детектирования, и имеет больший смысл изучать более новые модели.

**Сравнение с OverFeat**

OverFeat может рассматриваться как частный случай R-CNN, если в качестве регионов-кандидатов брать квадратные регионы нескольких разных размеров со всеми возможными смещениями. Однако OverFeat работает приблизительно в 9 раз быстрее R-CNN из-за того, что сверточные слои не запускаются заново для каждого региона-кандидата. Скорость работы R-CNN будет увеличена в последующих работах ([Fast R-CNN]($Fast R-CNN$), [Faster R-CNN]($Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks$)).